# WPI Data Cleaning & Validation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

### Load Raw WPI Dataset

Source: data.gov.in | 
Structure: Commodity-level indices with official weights


In [4]:
data_path = r"C:\Users\nithis kumar\OneDrive\Desktop\Projects\wpi-inflation-intelligence\dataset\wpi_index.csv"
wpi_raw = pd.read_csv(data_path)
wpi_raw.head()

,COMM_NAME,COMM_CODE,COMM_WT,INDX042012,INDX052012,INDX062012,INDX072012,INDX082012,INDX092012,INDX102012,...,INDX012023,INDX022023,INDX032023,INDX042023,INDX052023,INDX062023,INDX072023,INDX082023,INDX092023,INDX102023
0,Bajra,1101010104,0.08637,107.7,109.3,107.0,113.9,120.7,121.7,114.8,...,224.1,219.6,225.1,229.6,229.4,226.7,226.9,225.9,225.7,231.5
1,Moong,1101010203,0.07088,96.3,96.0,96.6,103.6,108.7,108.4,112.2,...,167.7,167.8,172.8,180.8,178.6,184.8,188.7,189.4,199.7,205.6
2,Masur,1101010204,0.05299,102.5,107.4,109.7,115.2,119.2,119.0,118.6,...,199.0,196.3,194.6,192.1,188.1,187.7,186.6,186.9,192.6,189.5
3,Peas/Chawali,1101010206,0.02444,113.2,123.1,134.3,147.4,143.3,153.8,146.6,...,160.4,160.2,166.9,195.5,199.5,196.0,215.9,219.8,227.4,237.6
4,Cauliflower,1101020108,0.16697,119.1,0.0,0.0,0.0,0.0,134.6,129.4,...,161.1,148.4,163.5,176.9,0.0,0.0,0.0,0.0,230.3,244.6


### Structural Validation

In [6]:
# Shape check - Dimensions(rowsxcolumns)
wpi_raw.shape

(869, 142)

In [16]:
# Expected columns
required_columns = {"COMM_NAME", "COMM_CODE", "COMM_WT"}
missing = required_columns - set(wpi_raw.columns)

if missing:
    raise ValueError(f"Required columns missing from wpi_raw: {missing}")

In [19]:
# Count index columns
index_columns = []
for i in wpi_raw.columns:
    if i.startswith("INDX"):
        index_columns.append(i)
len(index_columns)

139

In [26]:
wpi_raw.duplicated().sum()

np.int64(0)

Data structure is consistent.

### Missing Values

In [24]:
wpi_raw.isna().sum().sort_values(ascending=False).head()

COMM_WT       1
COMM_NAME     0
COMM_CODE     0
INDX042012    0
INDX052012    0
dtype: int64

There is a value missing in COMM_WT

Since the commodity weight is expected to be constant per commodity, fill it with the COMM_WT from same commodity code.

In [25]:
#keep the original for audit
wpi_raw["COMM_WT_ORIGINAL"] = wpi_raw["COMM_WT"]

In [37]:
wpi_raw["COMM_WT"] = (
    wpi_raw
    .groupby("COMM_CODE")["COMM_WT"]
    .transform("first")
)
wpi_raw["COMM_WT"] = (
    wpi_raw
    .groupby("COMM_CODE")["COMM_WT"]
    .transform("first")
)
wpi_raw.isna().sum().sort_values(ascending=False)

COMM_WT             1
COMM_WT_ORIGINAL    1
COMM_NAME           0
INDX042012          0
INDX052012          0
                   ..
INDX052023          0
INDX072023          0
INDX082023          0
INDX092023          0
INDX102023          0
Length: 143, dtype: int64

In [38]:
wpi_raw["COMM_CODE"].is_unique

True

In [42]:
wpi_raw[wpi_raw.isna().any(axis=1)]

,COMM_NAME,COMM_CODE,COMM_WT,INDX042012,INDX052012,INDX062012,INDX072012,INDX082012,INDX092012,INDX102012,...,INDX022023,INDX032023,INDX042023,INDX052023,INDX062023,INDX072023,INDX082023,INDX092023,INDX102023,COMM_WT_ORIGINAL
347,IV FOOD INDEX,2000000000,NaN,105.9,106.5,108.4,110.4,111.1,111.4,110.5,...,171.3,172.1,174.0,172.9,175.1,188.7,187.1,178.4,180.3,NaN


In [54]:
food_comm = wpi_raw.loc[wpi_raw["COMM_NAME"].str.contains("FOOD", case=False, na=False), ["COMM_NAME", "COMM_WT"]]
food_comm

,COMM_NAME,COMM_WT
18,f. OTHER FOOD ARTICLES,0.94799
54,n. Manufacture of processed ready to eat food,0.02428
162,"l. Manufacture of machinery for food, beverage...",0.22793
181,(A). FOOD ARTICLES,15.25585
198,c. OTHER NON-FOOD ARTICLES,1.96043
230,Instant Food/Prepared meals based on vegetables,0.02428
347,IV FOOD INDEX,NaN
348,a. FOOD GRAINS (CEREALS+PULSES),3.46238
660,Pressure vessel and tank for fermentation & ot...,0.05469
699,(B). NON-FOOD ARTICLES,4.11894


From observation, it's a structural Nan. Can't fill this one with mean/median. To Fill the Nan, the right method is to sum the total COMM_WT of its sub-components. 